In [4]:
import pandas as pd
DATA_DIR = 'data'
df_satmeta = pd.read_csv(f'{DATA_DIR}/raw/verification_no2_satellite_metadata.csv')

OOS_START_TIME = '2021-12-01'
OOS_END_TIME = '2022-02-28'

# no Taipei in OOS set
df_satmeta = df_satmeta[(df_satmeta.location.isin(['la','dl','dl-la-tpe'])) & (df_satmeta.time_start >= OOS_START_TIME) & (df_satmeta.time_end <= OOS_END_TIME)].copy()
df_satmeta['split'] = 'prod'

df_satmeta.to_csv(f'{DATA_DIR}/raw/prod_satellite_metadata.csv',index=False)

## Preprocessing

In [ ]:
##Extract NASADEM elevation data (Rerun required only if grid_metadata changes)
!python src/data/extract_nasadem.py --data_dir data

In [ ]:
##Extract OMI data from aws
!python src/data/extract_omi.py --data_dir data --stage prod --path_satmeta data/raw/prod_satellite_metadata.csv --s3url us_url --n_jobs 16 --cleanup


In [ ]:
##Extract TROPOMI data from aws
!python src/data/extract_tropomi.py --data_dir data --stage prod --path_satmeta data/raw/prod_satellite_metadata.csv --n_jobs 36 --s3url us_url --cleanup

In [ ]:
##Extract GFS
!python src/data/extract_gfs.py --data_dir data --stage prod --labels data/raw/verification_submission_format.csv --n_jobs 16 

In [ ]:
#Extract GEOS-CF assim/ hindcasts
!python src/data/extract_geos.py --data_dir data --stage prod --labels data/raw/verification_submission_format.csv --n_jobs 16 --cleanup

In [ ]:
##Create final datasets
!python src/data/create_dataset.py --data_dir data --stage prod --labels data/raw/verification_submission_format.csv --n_gfs 4 --n_jobs 4
!python src/data/create_dataset.py --data_dir data --stage prod --labels data/raw/verification_submission_format.csv --n_gfs 8 --n_jobs 4

## Inference

In [5]:
##Generate predictions 
!python src/inference.py --data_dir data --model_dir models --stage prod --subformat_path data/raw/verification_submission_format.csv --output_path prod_predictions.csv 

##### RUNNING INFERENCE FOR DATASET 4 ##########
location 1, fold: 0, samples:2306 2.416080049538193,86.68116929505692
location 1, fold: 1, samples:2306 3.1778854470227937,85.36251786057521
location 1, fold: 2, samples:2306 4.1313135452888154,86.85468343121879
location 1, fold: 3, samples:2306 3.62490460036303,84.23517508150746
location 1, fold: 4, samples:2306 2.3528884683674107,87.63833605879296
location 0, fold: 0, samples:1984 27.613664223250293,45.31313166518444
location 0, fold: 1, samples:1984 10.708023444303041,161.5020416167969
location 0, fold: 2, samples:1984 11.70420376058811,144.08035565742892
location 0, fold: 3, samples:1984 8.158849312689393,89.20481232783683
location 0, fold: 4, samples:1984 2.147182756286284,138.7456536657473
##### RUNNING INFERENCE FOR DATASET 8 ##########
location 1, fold: 0, samples:2306 3.344374875931946,87.10580686218174
location 1, fold: 1, samples:2306 3.6090656002898074,86.41092580792994
location 1, fold: 2, samples:2306 2.9837239585172224,88.

In [6]:
pred = pd.read_csv('prod_predictions.csv')
pred

,datetime,grid_id,value
0,2021-12-01T08:00:00Z,3A3IE,48.172999
1,2021-12-01T08:00:00Z,3S31A,61.153832
2,2021-12-01T08:00:00Z,7II4T,67.784498
3,2021-12-01T08:00:00Z,8BOQH,51.707428
4,2021-12-01T08:00:00Z,A2FBI,59.895220
...,...,...,...
4285,2022-02-27T08:00:00Z,WT52R,15.102103
4286,2022-02-27T08:00:00Z,X5DKW,48.900358
4287,2022-02-27T08:00:00Z,Z0VWC,40.340837
4288,2022-02-27T08:00:00Z,ZP1FZ,10.098105
